In [371]:
# # dependency installation
# !pip install typing_extensions==4.6.1 pydantic pydantic-core cuml umap-learn pandas numpy matplotlib scikit-learn nltk langdetect bertopic[spacy] openai
# !pip install bertopic flair

In [16]:
# Load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import download
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import json
from nltk.stem import WordNetLemmatizer
import os

from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from wordcloud import WordCloud
import umap
import matplotlib.pyplot as plt
import openai
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from bertopic import BERTopic  
from sklearn.metrics import silhouette_score

#find stop words, along with recipe realted stopwords
stop = stopwords.words('english')

FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"
FinancialPredicted = "Financial - Predicted"
LegalPredicted = "Legal - Predicted"
SchoolPredicted = "School - Predicted"
HealthPredicted = "Health - Predicted"


In [17]:
# Pile all comments into single array

fix = SchoolLabel
col = SchoolPredicted

data = pd.read_csv('final working dataset with only suicide articles.csv')
data = data[data[fix] > 0]
comments_all = data['Only Reason']
print(len(comments_all))
comments_all = comments_all.dropna()
print(len(comments_all))

834
834


In [18]:
comments_all = comments_all.to_list()
len(comments_all)

834

In [19]:
# seed topics
SEED_TOPICS_FLAG = False # seeds topics that model will attempt to converge to
    
# umap params
UMAP_N_NEIGHBOURS = 20
UMAP_N_COMPONENTS = 15
RANDOM_SEED = 42

# hdbscan params
MIN_CLUSTER_SIZE = 15

# ctfidf params
#REDUCE_FREQUENT_WORDS = False
REDUCE_FREQUENT_WORDS = True

# vectorizer params
STOP_WORDS_FLAG = True # stops words being used in topics, but not for embeddings
NGRAM_RANGE = (1,2)  

if STOP_WORDS_FLAG:
    STOP_WORDS = stopwords.words('english')
else: 
    STOP_WORDS = []

# representation model params
DIVERSITY = 0.1 # val in [0,1] specifying diversity of words in each topic

# outlier reduction params
OUTLIER_THRESH = 0.2

PRETRAINED_SENTENCE_MODEL = 'all-mpnet-base-v2'

In [20]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [21]:
sentence_model = SentenceTransformer(PRETRAINED_SENTENCE_MODEL)
embeddings = sentence_model.encode(comments_all, show_progress_bar=True)

Batches: 100%|██████████| 27/27 [00:28<00:00,  1.05s/it]


In [22]:
# init model
stopwords = list(stopwords.words('english')) + ['Legal Problem']
umap_model = umap.UMAP(n_neighbors=10, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=NGRAM_RANGE)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=REDUCE_FREQUENT_WORDS)
representation_model = KeyBERTInspired()

In [11]:
#for siz in range(6, 7):

#def func(topicsize, topicnum):
model = BERTopic(top_n_words = 10,
                    n_gram_range = (1,2),
                    language="english",
                    min_topic_size=6,
                    nr_topics=25,
                    calculate_probabilities = True,
                    embedding_model = PRETRAINED_SENTENCE_MODEL,
                    representation_model=representation_model,
                    vectorizer_model=vectorizer_model)

topics, probs = model.fit_transform(comments_all, embeddings)

cleaned_docs = model._preprocess_text(comments_all)
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topicsForCS = model.get_topics()
topicsForCS.pop(-1, None)
topic_words = [[word for word, _ in model.get_topic(topic) if word != ''] for topic in topicsForCS]
#topic_words = [[words for words, _ in model.get_topic(topic)] for topic in range(len(set(topicsForCS))-1)]
final_topic_words = []
for i in topic_words:
    if len(i) > 0:
        final_topic_words.append(i)

# Evaluate
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_v')
coherencev = coherence_model.get_coherence()

coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_npmi')
coherence = coherence_model.get_coherence()
# Calculate embeddings for your documents  
embedding1 = model._extract_embeddings(comments_all)  
# Calculate Silhouette score  
score = silhouette_score(embedding1, topics)  
print(" CV score: ",coherencev)
print(" C NPMI score: ",coherence)
print(" Silhouette score: ",score)
print(" Number of topics: ",len(set(topics))-1)
#return (coherencev, coherence, score, len(set(topics))-1)


# a1 = []
# a2 =[]
# a3 =[]
# a4 = []

# for a in range(len(topicsizes)):
#     for b in range(len(topic_num)):
#         coherencev, coherence, score, lentopics = func(topicsizes[a], topic_num[b])
#         a1.append(coherencev)
#         a2.append(coherence)
#         a3.append(score)
#         a4.append(lentopics)

tit = "All - " + fix
fig = model.visualize_barchart(top_n_topics=100, title = tit)
fig


 CV score:  0.4691156180273599
 C NPMI score:  -0.215789748125361
 Silhouette score:  0.04223867
 Number of topics:  24


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5544821619987488, 0.5593358278274536, 0.561004102230072,
                    0.6129833459854126, 0.688215434551239],
              'xaxis': 'x',
              'y': [suicide article  , bullying school  , suicide related  , cause
                    suicide  , suicide bullied  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.44926953315734863, 0.5090419054031372, 0.5140919089317322,
                    0.5291064977645874, 0.5465809106826782],
              'xaxis': 'x2',
              'y': [school setting  , shootings  , shooting incident  , occurred
                    school  , university killing  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4304717481136322, 0.4783586263656616, 0.47847944498062134,
                    0.5064784288406372, 0.6081482172012329],
              'xaxis': 'x3',
              'y': [students mental  , mental health  , committed suicide  ,
                    suicide  , suicide school  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4918835163116455, 0.5035029649734497, 0.5036143064498901,
                    0.5063598155975342, 0.666441023349762],
              'xaxis': 'x4',
              'y': [mass shooting  , committing suicide  , suicide  , suicide
                    carrying  , shootings schools  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.43623971939086914, 0.44106367230415344, 0.4557081162929535,
                    0.5353679656982422, 0.5752280950546265],
              'xaxis': 'x5',
              'y': [psychiatric  , shot people  , school shot  , shooting incident
                    , school shooting  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4561423361301422, 0.46859604120254517, 0.5017654895782471,
                    0.5188567638397217, 0.5248745083808899],
              'xaxis': 'x6',
              'y': [bullying  , committed suicide  , suicide  , bullying school  ,
                    school bullying  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5286605358123779, 0.5527504682540894, 0.6028838753700256,
                    0.6170554757118225, 0.6412495970726013],
              'xaxis': 'x7',
              'y': [reasons suicide  , student killed  , mass shooting  , prague
                    shooter  , shooter student  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.45956042408943176, 0.4614109694957733, 0.4707217514514923,
                    0.4823985993862152, 0.49522581696510315],
              'xaxis': 'x8',
              'y': [suicide attempts  , attempted suicide  , student change  ,
                    forced student  , developed suicidal  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.48598259687423706, 0.4862596392631531, 0.5530480146408081,
                    0.5840968489646912, 0.7411555647850037],
              'xaxis': 'x9',
              'y': [suicidal thoughts  , suicides article  , students mental  ,
                    pandemic school  , student suicides  ],
              'yaxis': 'y9'},
             {'marke

In [361]:
# model.reduce_topics(comments_all, nr_topics=20) 

In [364]:
model.visualize_barchart(top_n_topics=100, title = tit)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5602188110351562, 0.5607438087463379, 0.6374989748001099,
                    0.6440579891204834, 0.6816268563270569],
              'xaxis': 'x',
              'y': [suicide related  , reason suicide  , shootings schools  ,
                    suicide article  , school shooting  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5752385258674622, 0.6028537154197693, 0.6058910489082336,
                    0.6479189395904541, 0.6950048804283142],
              'xaxis': 'x2',
              'y': [cause suicide  , school bullying  , bullying school  ,
                    bullying article  , suicide bullied  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.45346423983573914, 0.46488654613494873, 0.4704318940639496,
                    0.523461103439331, 0.5243195295333862],
              'xaxis': 'x3',
              'y': [school problem  , suicide  , school life  , bullying school  ,
                    attempted suicide  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.48565414547920227, 0.489631325006485, 0.5134109258651733,
                    0.5274556279182434, 0.7142631411552429],
              'xaxis': 'x4',
              'y': [pandemic article  , school closures  , health crisis  ,
                    students mental  , student suicides  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5168226957321167, 0.5418113470077515, 0.5771273374557495,
                    0.5829818844795227, 0.5985344648361206],
              'xaxis': 'x5',
              'y': [suicide making  , mental health  , suicide school  , attempted
                    suicide  , suicide rates  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4702715277671814, 0.48702794313430786, 0.492289662361145,
                    0.5207138657569885, 0.6457201242446899],
              'xaxis': 'x6',
              'y': [problem article  , occurred school  , university killing  ,
                    shooting incident  , school shooting  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5526026487350464, 0.5540136098861694, 0.6043061017990112,
                    0.6424250602722168, 0.707125186920166],
              'xaxis': 'x7',
              'y': [shootings  , sandy hook  , educators article  , school newtown
                    , school shooting  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.44101670384407043, 0.46397191286087036, 0.4786455035209656,
                    0.49022769927978516, 0.5289105772972107],
              'xaxis': 'x8',
              'y': [category article  , health issues  , psychiatric  , problem
                    article  , mental health  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.540156364440918, 0.5534332394599915, 0.5586685538291931,
                    0.563895583152771, 0.604499101638794],
              'xaxis': 'x9',
              'y': [college athletes  , discusses suicides  , student athlete  ,
                    athletics academics  , student athletes  ],
              'yaxis': 'y9'},
       

In [344]:
model.get_representative_docs(15)

["The article mentions gunman Adam Lanza killed school shooting others This categorized 'School Problem suicide took place school setting The article also talks Alex Jones facing legal problems due conspiracy theories Sandy Hook Elementary School shooting Although Jones commit suicide legal problems facing direct result suicide incident hence 'Legal Problem category also applicable ``",
 "The article mentions Adam Lanza gunman responsible Sandy Hook Elementary School shooting killed school This falls 'School Problem category suicide occurred school setting related school shooting incident Additionally article also mentions Alex Jones sued victims families cited health problem reason defying court order Although Jones commit suicide mention health problem context legal case related suicide incident interpreted falling 'Health Problem category ``",
 "The article mentions Adam Lanza gunman responsible Sandy Hook Elementary School shooting killed school This falls 'School Problem category 

In [283]:
cleaned_docs = model._preprocess_text(comments_all)
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topicsForCS = model.get_topics()
topicsForCS.pop(-1, None)
topic_words = [[word for word, _ in model.get_topic(topic) if word != ''] for topic in topicsForCS]
#topic_words = [[words for words, _ in model.get_topic(topic)] for topic in range(len(set(topicsForCS))-1)]
final_topic_words = []
for i in topic_words:
    if len(i) > 0:
        final_topic_words.append(i)

# Evaluate
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_v')
coherencev = coherence_model.get_coherence()

coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_npmi')
coherence = coherence_model.get_coherence()
# Calculate embeddings for your documents  
embeddings = model._extract_embeddings(comments_all)  
# Calculate Silhouette score  
score = silhouette_score(embeddings, topics)  
print(" CV score: ",coherencev)
print(" C NPMI score: ",coherence)
print(" Silhouette score: ",score)

 CV score:  0.5024668853774736
 C NPMI score:  -0.21053739741528313
 Silhouette score:  -0.009728619


In [284]:
# Reduce outliers
#new_topics = model.reduce_outliers(comments_all, topics, probabilities=probs, strategy="probabilities")
#model.update_topics(comments_all, topics=new_topics)

In [365]:
model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0,
                                    'school shooting | suicide article | shootings schools | reason suicide | suicide related',
                                    78],
                                   [1,
                                    'suicide bullied | bullying article | bullying school | school bullying | cause suicide',
                                    74],
                                   [2,
                                    'attempted suicide | bullying school | school life | suicide | school problem',
                                    59],
                                   [3,
                                    'student suicides | students mental | health crisis | school closures | pandemic article',
                                    56],
                                   [4,
                                    'suicide rates | attempted suicide | suicide school | mental health | suicide making',
                                    43],
                                   [5,
                                    'school shooting | shooting incident | university killing | occurred school | problem article',
                                    42],
                                   [6,
                                    'school shooting | school newtown | educators article | sandy hook | shootings',
                                    32],
                                   [7,
                                    'mental health | problem article | psychiatric | health issues | category article',
                                    31],
                                   [8,
                                    'student athletes | athletics academics | student athlete | discusses suicides | college athletes',
                                    27],
                                   [9,
                                    'school shooting | school article | shooting incident | school officials | happened school',
                                    26],
                                   [10,
                                    'online bullying | bullying sexual | killed bullied | suicide becoming | suicide content',
                                    19],
                                   [11,
                                    'media children | effects social | harmful content | children article | impact social',
                                    17],
                                   [12,
                                    'issues university | mental health | suicide stanford | students mental | discriminating students',
                                    15],
                                   [13,
                                    'transgender individuals | trans advocates | lgbtq | among lgbtq | transgender nonbinary',
                                    14],
                                   [14,
                                    'prague shooter | shooter student | reasons suicide | categorize suicide | suicide categories',
                                    12],
                                   [15,
                                    'ramos gunman | gunman bullied | firearms despite | obtain firearms | guns',
                                    11],
                                   [16,
                                    'hazing experienced | attempted suicide | suicide attempt | hazing falls | hazing',
                                    11],
                                   [17,
                                    'student shot | student killed | shot campus | professors death | school shot',
                                    11],
                                   [18,
                                    'mass shooting | classmates shooting | student article | shooting killing | student saugus',
                                    10],
                                   [19,
                                    'student

In [366]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,207,-1_suicide article_school shooting_problem art...,NaN,[The article mentions school shooting incident...
1,0,78,0_school shooting_suicide article_shootings sc...,"[school shooting, suicide article, shootings s...",[The article mentions student Austin Lyle shot...
2,1,74,1_suicide bullied_bullying article_bullying sc...,"[suicide bullied, bullying article, bullying s...",[The article mentions 14-year-old Adriana Kuch...
3,2,59,2_attempted suicide_bullying school_school lif...,"[attempted suicide, bullying school, school li...",[The article discusses suicide 14-year-old gir...
4,3,56,3_student suicides_students mental_health cris...,"[student suicides, students mental, health cri...",[The article discusses mental health crisis am...
5,4,43,4_suicide rates_attempted suicide_suicide scho...,"[suicide rates, attempted suicide, suicide sch...",[The article mentions multiple instances indiv...
6,5,42,5_school shooting_shooting incident_university...,"[school shooting, shooting incident, universit...",[The article mentions gunman killed three stud...
7,6,32,6_school shooting_school newtown_educators art...,"[school shooting, school newtown, educators ar...",[The article mentions gunman killed 20 first-g...
8,7,31,7_mental health_problem article_psychiatric_he...,"[mental health, problem article, psychiatric, ...",[The article mentions Dylan Butler student Per...
9,8,27,8_student athletes_athletics academics_student...,"[student athletes, athletics academics, studen...",[The article discusses suicides several studen...


In [367]:
# embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
# model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/safetensors/school3", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 2 - pytorch
# model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/pytorch/school3", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# # Method 3 - pickle
# model.save("schoolbertopic3", serialization="pickle")

2024-04-05 12:11:00,921 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [23]:
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model = BERTopic.load("schoolbertopic3", embedding_model=embedding_model)

In [24]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,207,-1_suicide article_school shooting_problem art...,NaN,[The article mentions school shooting incident...
1,0,78,0_school shooting_suicide article_shootings sc...,"[school shooting, suicide article, shootings s...",[The article mentions student Austin Lyle shot...
2,1,74,1_suicide bullied_bullying article_bullying sc...,"[suicide bullied, bullying article, bullying s...",[The article mentions 14-year-old Adriana Kuch...
3,2,59,2_attempted suicide_bullying school_school lif...,"[attempted suicide, bullying school, school li...",[The article discusses suicide 14-year-old gir...
4,3,56,3_student suicides_students mental_health cris...,"[student suicides, students mental, health cri...",[The article discusses mental health crisis am...
5,4,43,4_suicide rates_attempted suicide_suicide scho...,"[suicide rates, attempted suicide, suicide sch...",[The article mentions multiple instances indiv...
6,5,42,5_school shooting_shooting incident_university...,"[school shooting, shooting incident, universit...",[The article mentions gunman killed three stud...
7,6,32,6_school shooting_school newtown_educators art...,"[school shooting, school newtown, educators ar...",[The article mentions gunman killed 20 first-g...
8,7,31,7_mental health_problem article_psychiatric_he...,"[mental health, problem article, psychiatric, ...",[The article mentions Dylan Butler student Per...
9,8,27,8_student athletes_athletics academics_student...,"[student athletes, athletics academics, studen...",[The article discusses suicides several studen...


In [14]:
topics = model.topics_
probs = model.probabilities_

# from existing predictions
left = data['Left'].to_list()
indsleft = []
for i in range(len(comments_all)):
    if left[i] == 1:
        indsleft.append(i)

alltopicsleft = [topics[i] for i in indsleft if i < len(topics)]
allprobsleft = [probs[i] for i in indsleft if i < len(probs)]
allcommentsleft = [comments_all[i] for i in indsleft if i < len(comments_all)]
onetextleft = " ".join(allcommentsleft)

right = data['Right'].to_list()
indsright = []
for i in range(len(comments_all)):
    if right[i] == 1:
        indsright.append(i)

alltopicsright = [topics[i] for i in indsright if i < len(topics)]
allprobsright = [probs[i] for i in indsright if i < len(probs)]
allcommentsright = [comments_all[i] for i in indsright if i < len(comments_all)]
onetextright = " ".join(allcommentsright)

regional = data['Regional'].to_list()
indsreg = []
for i in range(len(comments_all)):
    if regional[i] == 1:
        indsreg.append(i)

alltopicsreg = [topics[i] for i in indsreg if i < len(topics)]
allprobsreg = [probs[i] for i in indsreg if i < len(probs)]
allcommentsreg = [comments_all[i] for i in indsreg if i < len(comments_all)]
onetextreg = " ".join(allcommentsreg)

national = data['National'].to_list()
indsnat = []
for i in range(len(comments_all)):
    if national[i] == 1:
        indsnat.append(i)

alltopicsnat = [topics[i] for i in indsnat if i < len(topics)]
allprobsnat = [probs[i] for i in indsnat if i < len(probs)]
allcommentsnat = [comments_all[i] for i in indsnat if i < len(comments_all)]
onetextnat = " ".join(allcommentsnat)

embeddingsleft = sentence_model.encode(onetextleft, show_progress_bar=True)
embeddingsright = sentence_model.encode(onetextright, show_progress_bar=True)
embeddingsnat = sentence_model.encode(onetextnat, show_progress_bar=True)
embeddingsreg = sentence_model.encode(onetextreg, show_progress_bar=True)


onetopicleft, oneprobleft = model.transform(onetextleft)
onetopicright, oneprobright = model.transform(onetextright)
onetopicreg, oneprobreg = model.transform(onetextreg)
onetopicnat, oneprobnat = model.transform(onetextnat)

topicwiseprobleft = np.array(allprobsleft).mean(axis = 0)
topicwiseprobright = np.array(allprobsright).mean(axis = 0)
topicwiseprobnat = np.array(allprobsnat).mean(axis = 0)
topicwiseprobreg = np.array(allprobsreg).mean(axis = 0)

np.set_printoptions(suppress=True, formatter={'float': '{:0.5f}'.format})
print("Individual averaged: ", np.round(topicwiseprobleft - topicwiseprobright, 5))
indavgpol = np.round(topicwiseprobleft - topicwiseprobright, 5)
print("Combined 1 text: ", np.round(oneprobleft - oneprobright, 5))
combpol = np.round(oneprobleft - oneprobright, 5)

print("Individual averaged: ", np.round(topicwiseprobnat - topicwiseprobreg, 5))
print("Combined 1 text: ", np.round(oneprobnat - oneprobreg, 5))
indavgreg = np.round(topicwiseprobnat - topicwiseprobreg, 5)
combreg = np.round(oneprobnat - oneprobreg, 5)

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.29it/s]


Individual averaged:  [-0.00914 -0.02629 -0.01980 -0.04419 -0.00022 0.01181 0.01093 -0.02913
 0.01248 0.02065 -0.00201 0.02648 0.01328 0.00687 -0.04356 0.00941
 -0.00685 0.01315 -0.00030 -0.04127 0.00550 0.00894 0.00423 0.00843]
Combined 1 text:  [[-0.02632 -0.02654 -0.07830 -0.00440 -0.00794 -0.00935 -0.00971 -0.00624
  -0.00490 -0.00669 -0.02232 -0.00917 -0.00453 0.00350 -0.00317 -0.00315
  -0.00427 -0.00363 -0.00339 0.00345 -0.01356 -0.00343 -0.01790 -0.00304]]
Individual averaged:  [0.02747 0.03825 0.05155 -0.00238 -0.06243 -0.02205 -0.02134 -0.00937
 -0.00553 0.01529 0.01289 0.02000 -0.00066 -0.01041 0.00332 -0.00767
 -0.01050 0.00927 -0.01313 0.00051 -0.01366 -0.00093 -0.00983 -0.00422]
Combined 1 text:  [[-0.25003 0.00756 0.01680 0.00809 0.01238 -0.10110 -0.12024 -0.00825
  -0.00049 -0.07566 0.00768 0.00357 -0.00058 0.01608 -0.00265 -0.00185
  -0.00062 -0.00644 -0.01126 0.01626 0.00392 -0.00529 0.00485 -0.03074]]


In [25]:
ben = data.copy()

In [26]:
probabilities = model.probabilities_
print(probabilities.shape)
for i in range(probabilities.shape[1]):
    ben['Topic' + str(i)] = probabilities[:, i]


(834, 24)


In [27]:
ben.columns

Index(['Unnamed: 0', 'id', 'title', 'content', 'Publisher', 'Date Published',
       'Financial/Job Problem', 'School or Academic Related Problem',
       'Lack of Access to Health/Mental Health Care', 'Legal Problem', 'None',
       'Reasoning', 'Only Reason', 'National', 'Regional', 'Left', 'Right',
       'Fringe/Questionable', 'Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4',
       'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11',
       'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17',
       'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23'],
      dtype='object')

In [28]:
ben = ben.drop(columns=['Unnamed: 0','Reasoning'])

In [29]:
ben.columns

Index(['id', 'title', 'content', 'Publisher', 'Date Published',
       'Financial/Job Problem', 'School or Academic Related Problem',
       'Lack of Access to Health/Mental Health Care', 'Legal Problem', 'None',
       'Only Reason', 'National', 'Regional', 'Left', 'Right',
       'Fringe/Questionable', 'Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4',
       'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11',
       'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17',
       'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23'],
      dtype='object')

In [30]:
ben.to_csv('School problem articles topic model results.csv')

In [15]:
df = model.get_topic_info()

topicwiseprobl = np.insert(topicwiseprobleft, 0,-1)
topicwiseprobr = np.insert(topicwiseprobright, 0,-1)
topicwiseprobnat = np.insert(topicwiseprobnat, 0,-1)
topicwiseprobreg = np.insert(topicwiseprobreg, 0,-1)
df['Left - Right Individual Averaged'] = topicwiseprobl
df['Right - Individual Averaged'] = topicwiseprobr
df['National - Individual Averaged'] = topicwiseprobnat
df['Regional - Individual Averaged'] = topicwiseprobreg

oneprobl = np.insert(oneprobleft, 0,-1)
oneprobr = np.insert(oneprobright, 0,-1)
oneprobn = np.insert(oneprobnat, 0,-1)
oneprobre = np.insert(oneprobreg, 0,-1)
df['Left - One combined text'] = oneprobl
df['Right - One combined text'] = oneprobr
df['National - One combined text'] = oneprobn
df['Regional - One combined text'] = oneprobre

df['Left v/s Right - Individual Averaged'] = topicwiseprobl - topicwiseprobr
df['National v/s Regional - Individual Averaged'] = topicwiseprobnat - topicwiseprobreg
df['Left v/s Right - Combined Text'] = oneprobl - oneprobr
df['National v/s Regional - Combined Text'] = oneprobn - oneprobre

df.to_csv('school bertopic results.csv')

In [370]:
leftarticles = data[data['Left'] == 1]
rightarticles = data[data['Right'] == 1]

topicsleft, probsleft = model.transform(leftarticles['Only Reason'].to_list())
topicsright, probsright = model.transform(rightarticles['Only Reason'].to_list())

print(np.round(np.array(probsleft).mean(axis = 0) - np.array(probsright).mean(axis = 0), 5))

[0.01225 -0.02135 -0.00938 -0.00840 -0.00588 0.00923 0.02468 -0.01857
 0.00429 0.01503 -0.00440 0.02417 0.00438 -0.00089 -0.04843 0.00448
 -0.00159 0.00963 0.00094 -0.05502 -0.00234 0.00446 -0.00970 0.00170]
